# UnitTests: FiniteAlgebra

In [1]:
import unittest
from unittest import TestCase
from finite_algebras import *

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

ex = Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  16 example algebras are available.
  Use "Examples[INDEX]" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-groups.m

In [3]:
class TestFiniteAlgebra(TestCase):

    def setUp(self) -> None:
        
        # Magma Setup
        self.rps = ex[4]  # Magma
        
        # Semigroup Setup
        self.ex141_sg = ex[11]  # Semigroup
        
        # Group Setup
        self.z2 = generate_cyclic_group(2, name="Z2", description="Cyclic group of order 2")
        self.z4 = generate_cyclic_group(4, name="Z4", description="Cyclic group of order 4")
        self.s3 = generate_symmetric_group(3, name="S3", description="Symmetric group")
        self.ps3 = generate_powerset_group(3, "PS3", "Powerset group")
        self.v4 = ex[7]  # Klein-4 Group

        # Ring Setup
        self.psr2 = generate_powerset_ring(2, 'Powerset Ring 2', 'Ring on powerset of {0, 1}')
        self.r_mod_6 = generate_algebra_mod_n(6)  # Ring
        self.f_mod_5 = generate_algebra_mod_n(5)  # Field
        
    # Magma Tests

    def test_elements(self):
        self.assertEqual(self.rps.elements, ['r', 'p', 's'])

    def test_table(self):
        self.assertEqual(self.rps.table, CayleyTable([[0, 1, 0], [1, 1, 2], [0, 2, 2]]))

    def test_rps_is_not_associative(self):
        rp = self.rps.op('r', 'p')
        ps = self.rps.op('p', 's')
        rp_s = self.rps.op(rp, 's')
        r_ps = self.rps.op('r', ps)
        self.assertNotEqual(rp_s, r_ps)

    def test_table_with_names(self):
        self.assertEqual(self.rps.table.to_list_with_names(self.rps.elements),
                         [['r', 'p', 'r'], ['p', 'p', 's'], ['r', 's', 's']])

    def test_is_associative(self):
        self.assertEqual(self.rps.is_associative(), False)

    def test_is_commutative(self):
        self.assertEqual(self.rps.is_commutative(), True)

    def test_identity(self):
        self.assertIsNone(self.rps.identity)

    def test_make_finite_algebra_1(self):
        rps2 = make_finite_algebra('RPS', "Rock, Paper, Scissors",
                                   ['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])
        self.assertEqual(rps2, self.rps)


    # Semigroup Tests

    def test_rps_is_associative(self):
        self.assertEqual(self.ex141_sg.is_associative(), True)

    def test_rps_is_not_commutative(self):
        self.assertEqual(self.ex141_sg.is_commutative(), False)

    def test_rps_has_no_identity(self):
        self.assertIsNone(self.ex141_sg.identity)

    def test_ex141_is_associative(self):
        ab = self.ex141_sg.op('a', 'b')
        bc = self.ex141_sg.op('b', 'c')
        ab_c = self.ex141_sg.op(ab, 'c')
        a_bc = self.ex141_sg.op('a', bc)
        self.assertEqual(ab_c, a_bc)

    def test_fail_to_create_semigroup_from_rps(self):
        """The RPS magma can't also be made into a semigroup."""
        with self.assertRaises(ValueError):
            Semigroup('rps', 'foobar', ['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])


    # Group Tests
    
    def test_elements_accessor_ps3(self):
        self.assertEqual(self.ps3.elements, ['{}', '{0}', '{1}', '{2}', '{0, 1}',
                                             '{0, 2}', '{1, 2}', '{0, 1, 2}'])

    def test_table_accessor_ps3(self):
        self.assertEqual(self.ps3.table, CayleyTable([[0, 1, 2, 3, 4, 5, 6, 7],
                                                      [1, 0, 4, 5, 2, 3, 7, 6],
                                                      [2, 4, 0, 6, 1, 7, 3, 5],
                                                      [3, 5, 6, 0, 7, 1, 2, 4],
                                                      [4, 2, 1, 7, 0, 6, 5, 3],
                                                      [5, 3, 7, 1, 6, 0, 4, 2],
                                                      [6, 7, 3, 2, 5, 4, 0, 1],
                                                      [7, 6, 5, 4, 3, 2, 1, 0]]))

    def test_is_associative_ps3(self):
        self.assertEqual(self.ps3.is_associative(), True)

    def test_is_commutative_ps3(self):
        self.assertEqual(self.ps3.is_commutative(), True)

    def test_identity_accessor_ps3(self):
        self.assertEqual(self.ps3.identity, '{}')

    def test_elements_accessor_z4(self):
        self.assertEqual(self.z4.elements, ['e', 'a', 'a^2', 'a^3'])

    def test_table_accessor_z4(self):
        self.assertEqual(self.z4.table, CayleyTable([[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]))

    def test_is_associative_z4(self):
        self.assertEqual(self.z4.is_associative(), True)

    def test_is_commutative_z4(self):
        self.assertEqual(self.z4.is_commutative(), True)

    def test_identity_accessor_z4(self):
        self.assertEqual(self.z4.identity, 'e')

    def test_elements_accessor_s3(self):
        self.assertEqual(self.s3.elements, ['(1, 2, 3)', '(1, 3, 2)', '(2, 1, 3)',
                                            '(2, 3, 1)', '(3, 1, 2)', '(3, 2, 1)'])

    def test_table_accessor_s3(self):
        self.assertEqual(self.s3.table, CayleyTable([[0, 1, 2, 3, 4, 5],
                                                     [1, 0, 4, 5, 2, 3],
                                                     [2, 3, 0, 1, 5, 4],
                                                     [3, 2, 5, 4, 0, 1],
                                                     [4, 5, 1, 0, 3, 2],
                                                     [5, 4, 3, 2, 1, 0]]
                                                    ))

    def test_is_associative_s3(self):
        self.assertEqual(self.s3.is_associative(), True)

    def test_is_commutative_s3(self):
        self.assertEqual(self.s3.is_commutative(), False)

    def test_identity_accessor_s3(self):
        self.assertEqual(self.s3.identity, '(1, 2, 3)')

    def test_direct_product_and_isomorphic(self):
        z2_sqr = self.z2 * self.z2
        self.assertEqual(self.v4.isomorphic(z2_sqr),
                         {'e': 'e:e', 'h': 'e:a', 'v': 'a:e', 'r': 'a:a'})

    def test_not_isomorphic(self):
        self.assertFalse(self.z4.isomorphic(self.v4))


    # Ring & Field Tests

    def test_ring_or_field_elements(self):
        self.assertEqual(self.psr2.elements, ['{}', '{0}', '{1}', '{0, 1}'])
        self.assertEqual(self.r_mod_6.elements, ['a0', 'a1', 'a2', 'a3', 'a4', 'a5'])
        self.assertEqual(self.f_mod_5.elements, ['a0', 'a1', 'a2', 'a3', 'a4'])

    def test_powerset_mult_table(self):
        self.assertEqual(self.psr2.mult_table, CayleyTable([[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 2, 2], [0, 1, 2, 3]]))
        self.assertEqual(self.r_mod_6.mult_table, CayleyTable([[0, 0, 0, 0, 0, 0],
                                                               [0, 1, 2, 3, 4, 5],
                                                               [0, 2, 4, 0, 2, 4],
                                                               [0, 3, 0, 3, 0, 3],
                                                               [0, 4, 2, 0, 4, 2],
                                                               [0, 5, 4, 3, 2, 1]]))
        self.assertEqual(self.f_mod_5.mult_table, CayleyTable([[0, 0, 0, 0, 0],
                                                               [0, 1, 2, 3, 4],
                                                               [0, 2, 4, 1, 3],
                                                               [0, 3, 1, 4, 2],
                                                               [0, 4, 3, 2, 1]]))

    def test_add_identity(self):
        self.assertEqual(self.psr2.add_identity, '{}')
        self.assertEqual(self.r_mod_6.add_identity, 'a0')
        self.assertEqual(self.f_mod_5.add_identity, 'a0')

    def test_add(self):
        self.assertEqual(self.psr2.add('{1}', '{0, 1}'), '{0}')
        self.assertEqual(self.r_mod_6.add('a1', 'a2'), 'a3')
        self.assertEqual(self.f_mod_5.add('a1', 'a2'), 'a3')

    def test_mult(self):
        self.assertEqual(self.psr2.mult('{1}', '{0, 1}'), '{1}')
        self.assertEqual(self.r_mod_6.mult('a2', 'a3'), 'a0')
        self.assertEqual(self.f_mod_5.mult('a2', 'a3'), 'a1')

    def test_mult_identity(self):
        self.assertEqual(self.psr2.mult_identity, '{0, 1}')
        self.assertEqual(self.r_mod_6.mult_identity, 'a1')
        self.assertEqual(self.f_mod_5.mult_identity, 'a1')

    def test_has_mult_identity(self):
        self.assertTrue(self.psr2.has_mult_identity())
        self.assertTrue(self.r_mod_6.has_mult_identity())
        self.assertTrue(self.f_mod_5.has_mult_identity())

    def test_ring_or_field_zero_divisors(self):
        self.assertEqual(self.psr2.zero_divisors(), ['{0}', '{1}'])
        self.assertEqual(self.r_mod_6.zero_divisors(), ['a2', 'a3', 'a4'])
        self.assertEqual(self.f_mod_5.zero_divisors(), [])

In [4]:
unittest.main(argv=[''], verbosity=3, exit=False)

test_add (__main__.TestFiniteAlgebra) ... ok
test_add_identity (__main__.TestFiniteAlgebra) ... ok
test_direct_product_and_isomorphic (__main__.TestFiniteAlgebra) ... ok
test_elements (__main__.TestFiniteAlgebra) ... ok
test_elements_accessor_ps3 (__main__.TestFiniteAlgebra) ... ok
test_elements_accessor_s3 (__main__.TestFiniteAlgebra) ... ok
test_elements_accessor_z4 (__main__.TestFiniteAlgebra) ... ok
test_ex141_is_associative (__main__.TestFiniteAlgebra) ... ok
test_fail_to_create_semigroup_from_rps (__main__.TestFiniteAlgebra)
The RPS magma can't also be made into a semigroup. ... ok
test_has_mult_identity (__main__.TestFiniteAlgebra) ... ok
test_identity (__main__.TestFiniteAlgebra) ... ok
test_identity_accessor_ps3 (__main__.TestFiniteAlgebra) ... ok
test_identity_accessor_s3 (__main__.TestFiniteAlgebra) ... ok
test_identity_accessor_z4 (__main__.TestFiniteAlgebra) ... ok
test_is_associative (__main__.TestFiniteAlgebra) ... ok
test_is_associative_ps3 (__main__.TestFiniteAlgebra) 